To write a TensorFlow program based on pre-made Estimators, you must perform the following tasks:

+ Create one or more input functions.
+ Define the model's feature columns.
+ Instantiate an Estimator, specifying the feature columns and various hyperparameters.
+ Call one or more methods on the Estimator object, passing the appropriate input function as the source of the data

In [ ]:
import tensorflow as tf
import pandas as pd
# https://www.tensorflow.org/get_started/premade_estimators

In [ ]:
ls data

In [1]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [2]:
def load_data(y_name='Species'):
    """Returns the iris data set as (train_x, train_y), (test_x, test_y)"""
    
    # Define the paths to the data training and test sets.
    train_path = 'data/iris_training.csv'
    test_path = 'data/iris_test.csv'
    
    # Create the dataframes.
    train = pd.read_csv(
        train_path,
        names=CSV_COLUMN_NAMES,
        header=0)
    train_x, train_y = train, train.pop(y_name)
    
    test = pd.read_csv(
        test_path,
        names=CSV_COLUMN_NAMES,
        header=0)
    test_x, test_y = test, test.pop(y_name)
    
    return (train_x, train_y), (test_x, test_y)

In [5]:
# load_data()

In [6]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset

In [37]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    print(dataset)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
    print(dataset)

    # Return the dataset.
    return dataset

**batch**
The set of examples used in one iteration (that is, one gradient update) of model training.

**batch size**
The number of examples in a batch. For example, the batch size of SGD is 1, while the batch size of a mini-batch is usually between 10 and 1000. Batch size is usually fixed during training and inference; however, TensorFlow does permit dynamic batch sizes.

In [38]:
args = {
    'batch_size': 10,
    'train_steps': 100,
}

## Defining Feature Columns

In [39]:
# Load the data.
(train_x, train_y), (test_x, test_y) = load_data()

In [40]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

## Instantiate an estimator


In [41]:
# Build a DNN with 2 hidden layers and 10 nodes in each hidden layer.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpc6uvqelt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fadfb4505c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [42]:
# Train the Model.
classifier.train(
    input_fn=lambda:train_input_fn(
        train_x,
        train_y,
        args['batch_size']),
    steps=args['train_steps'])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpc6uvqelt/model.ckpt.
INFO:tensorflow:loss = 47.965073, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmpc6uvqelt/model.ckpt.
INFO:tensorflow:Loss for final step: 3.1549864.


## Evaluate the trained model

In [43]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, args['batch_size']))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

<TensorSliceDataset shapes: ({SepalLength: (), SepalWidth: (), PetalLength: (), PetalWidth: ()}, ()), types: ({SepalLength: tf.float64, SepalWidth: tf.float64, PetalLength: tf.float64, PetalWidth: tf.float64}, tf.int64)>
<BatchDataset shapes: ({SepalLength: (?,), SepalWidth: (?,), PetalLength: (?,), PetalWidth: (?,)}, (?,)), types: ({SepalLength: tf.float64, SepalWidth: tf.float64, PetalLength: tf.float64, PetalWidth: tf.float64}, tf.int64)>
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-03-20-04:58:20
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpc6uvqelt/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-03-20-04:58:20
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.9, average_loss = 0.27815613, global_step = 100, loss = 2.7815611

Test set accuracy: 0.900



In [44]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
    input_fn=lambda:eval_input_fn(predict_x, None, batch_size=args['batch_size']))

In [45]:
for pred_dict, expec in zip(predictions, expected):
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(SPECIES[class_id],
                          100 * probability, expec))

<TensorSliceDataset shapes: {SepalLength: (), SepalWidth: (), PetalLength: (), PetalWidth: ()}, types: {SepalLength: tf.float32, SepalWidth: tf.float32, PetalLength: tf.float32, PetalWidth: tf.float32}>
<BatchDataset shapes: {SepalLength: (?,), SepalWidth: (?,), PetalLength: (?,), PetalWidth: (?,)}, types: {SepalLength: tf.float32, SepalWidth: tf.float32, PetalLength: tf.float32, PetalWidth: tf.float32}>
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpc6uvqelt/model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (96.2%), expected "Setosa"

Prediction is "Versicolor" (83.2%), expected "Versicolor"

Prediction is "Versicolor" (61.3%), expected "Virginica"
